# Q&A with the Compettitive Intelligence Agent

This notebook demonstrates how to perform business analysis of companies using a multi-agent system.

## 1. Setup

In [ ]:
import boto3
import botocore
import datetime
import json
import logging
import uuid

# getting boto3 clients for required AWS services
session = boto3.session.Session()
sts_client = session.client("sts")
iam_client = session.client("iam")
lambda_client = session.client("lambda")
bedrock_agent_client = session.client("bedrock-agent")
bedrock_agent_runtime_client = session.client("bedrock-agent-runtime")
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
print(f"{region=}\n{account_id=}")

## 2. Define Helper Functions

In [ ]:
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

def get_hcls_agent(
    name_starts_with: str, bedrock_agent_client: botocore.client
) -> list[dict]:
    """
    Get the list of agents whose name starts with the given string.
    Uses pagination to handle large result sets.
    """
    agent_info = []
    paginator = bedrock_agent_client.get_paginator('list_agents')
    
    # Iterate through each page of results
    for page in paginator.paginate():
        # Process each page of results
        for agent in page.get("agentSummaries", []):
            if agent["agentName"].startswith(name_starts_with):
                agent_info.append({"name": agent["agentName"], "id": agent["agentId"]})
    
    return agent_info



def get_agent_alias(agent_id: str, bedrock_agent_client: botocore.client) -> list[dict]:
    """
    Get the list of aliases for the given agent.
    Uses pagination to handle large result sets.
    """
    alias_info = []
    paginator = bedrock_agent_client.get_paginator('list_agent_aliases')
    
    # Iterate through each page of results
    for page in paginator.paginate(agentId=agent_id):
        # Process each page of results
        for alias in page.get("agentAliasSummaries", []):
            alias_info.append({"name": alias["agentAliasName"], "id": alias["agentAliasId"]})
    
    return alias_info


def invoke_agent_helper(
    query, session_id, agent_id, alias_id, enable_trace=False, session_state=None, end_session=False
):
    if not session_state:
        session_state = {"promptSessionAttributes": {"today": str(datetime.date.today())}}

    # invoke the agent API
    agent_response = bedrock_agent_runtime_client.invoke_agent(
        inputText=query,
        agentId=agent_id,
        agentAliasId=alias_id,
        sessionId=session_id,
        enableTrace=enable_trace,
        endSession=end_session,
        sessionState=session_state
    )

    event_stream = agent_response['completion']
    try:
        for event in event_stream:
            if 'chunk' in event:
                data = event['chunk']['bytes']
                if enable_trace:
                    logger.info(f"Final answer ->\n{data.decode('utf8')}")
                agent_answer = data.decode('utf8').strip()
                logger.info(agent_answer)
                return None
                # End event indicates that the request finished successfully
            elif 'trace' in event:
                if enable_trace:
                    logger.info(json.dumps(event['trace']['trace'], indent=2))
            else:
                raise Exception("unexpected event.", event)
    except Exception as e:
        raise Exception("unexpected event.", e)



## 3. Get Agent Details

In [ ]:
available_agents = get_hcls_agent("Competitive-Intelligence-Agent", bedrock_agent_client)
current_agent = available_agents[0]
print(f"Current agent name: {current_agent['name']}")
print(f"Current agent id: {current_agent['id']}")

current_agent_alias = get_agent_alias(current_agent['id'], bedrock_agent_client)[0]
print(f"Current agent alias: {current_agent_alias['id']}")

session_id:str = str(uuid.uuid1())
agent_id = current_agent['id']
agent_alias_id = current_agent_alias['id']

## 4. Chat with Agent

In [ ]:
invoke_agent_helper(
    "Please write a competitive intelligence report on Eli Lilly, focusing on recent news about their R&D pipeline and any patent filings in the past 200 days related to tirzepatide or zepbound.",
    session_id,
    agent_id,
    agent_alias_id,
    enable_trace=True,
)